In [79]:
import cv2
import re
import os
import numpy as np

In [80]:
nKeyPointsInitial = 5000

imageSetNumber = 1
imageSetPath = "./In-Sample/"+str(imageSetNumber)+"/"


In [81]:
images = [ cv2.imread((imageSetPath+image)) for image in os.listdir(imageSetPath)]

for i in range(len(images)) :
    images[i] = cv2.cvtColor(images[i], cv2.COLOR_RGB2GRAY)

keypoints = []
descriptors = []

# print(images)

In [82]:
# img = (cv2.imread(images[0]))
# img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# cv2.imshow('img0', images[0])
# cv2.imshow('img1', images[1])
# cv2.imshow('img2', images[2])

# if cv2.waitKey(0) & 0xFF == ord('q'):
#     cv2.destroyAllWindows()

In [83]:
# sift = cv2.xfeatures2d.SIFT_create()
# surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=nKeyPointsInitial)
# fast = cv2.FastFeatureDetector_create(nKeyPointsInitial) 
# brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()



# keypoints_sift, descriptors = sift.detectAndCompute(img, None)
# keypoints_surf, descriptors = surf.detectAndCompute(img, None)


for image in images :
    keypoints_orb, descriptors_orb = orb.detectAndCompute(image, None)
    keypoints.append(keypoints_orb)
    descriptors.append(descriptors_orb)


# keypoints_brief, descriptors_brief = brief.compute(img, None)
# keypoints_fast = fast.detect(img, None)




# img = cv2.drawKeypoints(img, keypoints_fast, None)
# cv2.imwrite("1_1-out.jpg",img)
# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [84]:
# Create a Brute Force Matcher object.
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

# Perform the matching between the BRIEF descriptors of the training image and the test image
matches = bf.match(descriptors[1], descriptors[2])

# The matches with shorter distance are the ones we want.
matches = sorted(matches, key = lambda x : x.distance)

result = cv2.drawMatches(images[1], keypoints[1], images[2], keypoints[2], matches, images[1], flags = 2)

# cv2.imshow('res',result)
# cv2.waitKey(0)
# cv2.imwrite("log.jpg",result)


In [85]:

src_pts = np.float32([ keypoints[1][m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ keypoints[2][m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

# matchesMask = mask.ravel().tolist()


# h,w = images[1].shape
# pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
# dst = cv2.perspectiveTransform(pts,M)


# cv2.imwrite("log1.jpg",images[2])

# images[2] = cv2.polylines(images[2],[np.int32(dst)],True,255,3, cv2.LINE_AA)


# cv2.imwrite("log2.jpg",images[2])


In [93]:
def warpTwoImages(img1, img2, H):
    '''warp img2 to img1 with homograph H'''
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate

    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
    return result



src_pts = np.float32([ keypoints[1][m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ keypoints[2][m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

result = warpTwoImages(images[2], images[1], M)

In [94]:
# Apply panorama correction

# width = images[1].shape[1] + images[2].shape[1]
# height = images[1].shape[0] + images[2].shape[0]
# result = cv2.warpPerspective(images[1], M, (width, height))
# result[0:images[2].shape[0], 0:images[2].shape[1]] = images[2]

cv2.imwrite("log2.jpg",result)
# cv2.imwrite("log1.jpg",images[1])




True